In [1]:
from bs4 import BeautifulSoup
import os
from function import read_list_webs, crawl_list_news, crawl_news_text

In [2]:
list_webs = read_list_webs(os.path.join(".", "list_webs.txt"))

In [3]:
for line in list_webs:
    print(line)

https://hanoi.fpt.edu.vn/360-do-hoa-lac
https://vnexpress.net/chu-de/dai-hoc-fpt-2161


In [4]:
page = 0

In [8]:
res = []
for web in list_webs:
    links = crawl_list_news(web)
    print(len(links))
    for link in links: 
        res.append(link)
print(res)
print(len(res))
with open("text.txt", "w", encoding='utf-8') as f:
    for link in res:
        f.write(str(link)+'\n')
    f.close()

240
400
['https://hanoi.fpt.edu.vn/mua-he-ruc-ro-chuong-trinh-tieng-anh-chuyen-sau-level-6-tai-philippines-va-malaysia.html', 'https://hanoi.fpt.edu.vn/gap-go-giang-vien-dai-hoc-fpt-gianh-huy-chuong-vang-tai-giai-muay-the-gioi.html', 'https://hanoi.fpt.edu.vn/nhom-sinh-vien-dai-hoc-fpt-phat-trien-san-pham-ai-phat-hien-hanh-vi-bat-thuong-bang-camera-an-ninh.html', 'https://hanoi.fpt.edu.vn/cuu-sinh-vien-dai-hoc-fpt-gianh-hoc-bong-tien-si-tai-phap.html', 'https://hanoi.fpt.edu.vn/doc-dao-do-an-thiet-ke-bo-nhan-dien-thuong-hieu-cho-lien-doan-xiec-viet-nam-cua-sinh-vien-dai-hoc-fpt.html', 'https://hanoi.fpt.edu.vn/do-an-cua-sinh-vien-nganh-tri-tue-nhan-tao-duoc-chap-thuan-tai-hoi-thao-khoa-hoc-quoc-te-icaisc-2023.html', 'https://hanoi.fpt.edu.vn/vong-chung-ket-fpt-edu-nihongoeng-2023-duoc-to-chuc-tai-nhat-ban.html', 'https://hanoi.fpt.edu.vn/nhung-loi-the-khi-so-huu-ielts-6-0-tro-len-tai-dai-hoc-fpt.html', 'https://hanoi.fpt.edu.vn/mua-he-ruc-ro-chuong-trinh-tieng-anh-chuyen-sau-level-6-ta